# A python solution to calculate Pv for pure substances using PR-EOS

***
**Note: Click on the `Cell` in the task bar above, then click on `Run All` to run the entire code**


In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown
import math

## The cells below solves question 1
Calculate the vapor pressure of propane together with the densities of the liquid and gas at 104°F.
Compare your answers with values from Figures 2-7, 2-12, and 3-4 (McCain Jr., W.D. The
Properties of Petroleum Fluids (2nd Edition). PennWell Publishing Company, Tulsa, OK, 1990.). 

### Utility functions

In [9]:
# utility methods
def convert_f_to_r(temp):
    return temp + 459.67

def calculate_tr(temp, temp_c):
    converted_temp = convert_f_to_r(temp)
    converted_temp_c = convert_f_to_r(temp_c)
    return converted_temp / converted_temp_c

def calculate_b(temp_c, pressure_c):
    converted_temp_c = convert_f_to_r(temp_c)
    return (0.07780 * ((10.732 * converted_temp_c) / (pressure_c)))

def calculate_ac(temp_c, pressure_c):
    converted_temp_c = convert_f_to_r(temp_c)
    return (0.45724 * ((10.732 ** 2)* (converted_temp_c ** 2)) / pressure_c)

def calculate_alpha(w, temp, temp_c):
    tr = calculate_tr(temp, temp_c)
    alpha_half = 1 + (0.37464 + (1.54226 * w) - (0.26992 * (w ** 2))) * (1 - (tr ** 0.5))
    return alpha_half ** 2

def calculate_at(ac, alpha):
    return ac * alpha

# Trial section functions
def calculate_A(aT, trial_pressure, sys_temp):
    temp = convert_f_to_r(sys_temp)
    return (aT * trial_pressure) / ((10.732 ** 2) * (temp ** 2))

def calculate_B(b, trial_pressure, sys_temp):
    temp = convert_f_to_r(sys_temp)
    return (b * trial_pressure) / (10.732 * temp)

def cubic_root_calculation_for_PR_EOS(A, B):
    # define coefficents
    a = 1
    b = -(1 - B)
    c = (A - (2*B) - (3*(B ** 2)))
    d = -((A*B) - (B**2) - (B**3))
    
    coefs = [a, b, c, d]
    roots = np.roots(coefs)
    # check for complex numbera and get the real part
    new_roots = []
    for number in roots:
        new_roots.append(number.real)
    
    return new_roots

def get_highest_and_lowest_root(roots):
    try:
        if type(roots) != list:
            raise Exception('The roots have to be a list')
        else:
            lowest = min(roots)
            highest = max(roots)
            return { "zg": highest, "zl": lowest }
    except Exception as error:
        text = f"<div style='color:red'>{error}</div>"
        display(HTML(text))
            

def solve_for_fg_fl(z, B, A, trial_pressure):
    z = z
    B = B
    A = A
    p = trial_pressure
    first_part = math.log(z - B)
    second_part = (A / ((2**1.5)*B))
    numerator = (z + (((2**0.5) + 1) * B))
    denominator = (z - (((2**0.5) - 1) * B))
    log_part = math.log(numerator / denominator)
    f_value = math.exp(z - 1 - first_part - (second_part * log_part)) * trial_pressure
    
    return f_value

def render_table(data, title, transpose=False, change_indexing_id=False, new_index="S/N", should_display=True):
    df = pd.DataFrame(data)
    if change_indexing_id:
        df.set_index(new_index, inplace=True)
    if should_display:
        display(df)
    return df

### Working data for question one
This will contain the list of species to be considered, their critical properties and acentric factor

In [10]:
# working data question 1
species_list = {
    "1": { "name": "iso-butane", "pc": 527.9, "tc": 274.46, "w": 0.1852 },
    "2": { "name": "propane", "pc": 616.0, "tc": 206.06, "w": 0.1522 }
}

### Widgets and interaction

In [11]:
# Create some widgets
title = widgets.HTML(
    value="<h2>Select the species and input the temperature</h2>"
)
temperature = widgets.FloatText(
    value="104", 
    description="Farenheit", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)
pressure_widget = widgets.FloatText(
    value="100", 
    min=14.7, 
    description="psia", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

# loop through to key for the dropdown options
dropdown_options = [(value["name"], int(key)) for key, value in species_list.items()]

# specie dropdown
specie_dropdown = widgets.Dropdown(
    options=dropdown_options,
    value=2,
    description='Specie', 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

# interact with widgets
def on_value_change(sys_temp, specie, initial_trial_pressure):
    selected_specie = species_list[f"{specie}"]
    pc = selected_specie['pc']
    tc = selected_specie['tc']
    tr = round(calculate_tr(sys_temp, tc), 5)
    b = round(calculate_b(tc, pc), 4)
    ac = round(calculate_ac(tc, pc), 0)
    w = selected_specie['w']
    alpha = round(calculate_alpha(w, sys_temp, tc), 4)
    aT = round(calculate_at(ac, alpha), 0)
    
#     create table for the initially calculated values
    initial_calulated_values = {
        "Tr": [tr],
        "b": [b],
        "ac": [ac],
        "alpha": [alpha],
        "at": [aT]
    }
    
    render_table(initial_calulated_values, 'PR-EOS REQUIRED COEFFICIENTS')
    
#     iteration code
    iteration_table_data = {
        "z1": [],
        "z2": [],
        "z3": [],
        "zg": [],
        "zl": [],
        "fg": [],
        "fl": [],
        "fg - fl": [],
        "pressure": [],
        "Vmg": [],
        "Vml": [],
    }
    
    fg = 0
    fl = 1000
    pressure = initial_trial_pressure
    tolerance = 0.1
    while abs(fg - fl) >= tolerance:
        A = round(calculate_A(aT, pressure, sys_temp), 5)
        B = round(calculate_B(b, pressure, sys_temp), 6)
        roots = cubic_root_calculation_for_PR_EOS(A, B)

        iteration_table_data["z1"].append(roots[0])
        iteration_table_data["z2"].append(roots[1])
        iteration_table_data["z3"].append(roots[2])

        z_values = get_highest_and_lowest_root(roots)
        iteration_table_data["zg"].append(z_values["zg"])
        iteration_table_data["zl"].append(z_values["zl"])

        fg = solve_for_fg_fl(z_values["zg"], B, A, pressure)
        fl = solve_for_fg_fl(z_values["zl"], B, A, pressure)
        iteration_table_data["fg"].append(fg)
        iteration_table_data["fl"].append(fl)
        
        iteration_table_data["pressure"].append(pressure)
        iteration_table_data["fg - fl"].append(abs(fg - fl))
        
        Vmg = (z_values["zg"] * 10.732 * convert_f_to_r(sys_temp)) / pressure
        Vml = (z_values["zl"] * 10.732 * convert_f_to_r(sys_temp)) / pressure
        iteration_table_data["Vmg"].append(Vmg)
        iteration_table_data["Vml"].append(Vml)
        
        pressure = pressure + 0.01
        
    iteration_table = render_table(iteration_table_data, "ITERATION RESULTS")
    render_table(iteration_table.tail(1), 'FINAL RESULTS', True)
    
    
# Display widgets    
display(title)
trap_return = widgets.interact(on_value_change, sys_temp=temperature, specie=specie_dropdown, initial_trial_pressure=pressure_widget)

HTML(value='<h2>Select the species and input the temperature</h2>')

interactive(children=(FloatText(value=104.0, description='Farenheit', layout=Layout(margin='10px 0px 10px 10px…

## The cells below solves question 2

Calculate the compositions and densities of the equilibrium liquid and gas of the mixture given
below at 160°F and 2000 psia. Use binary interaction coefficients of 0.021 for methane-n-butane,
0.032 for methane-n-decane, and 0.0 for n-butane-n-decane. The pre-specified tolerance is for 0.098.
Note: The first trial must be done manually; then use your own coded program to perform the
necessary iterations. 

|Component| Composition, mole fraction|
|----|----|
|Methane| 0.5523|
|n-Butane| 0.3630|
|n-Decane| 0.0838|
||1.0000| 

### Working data for question 2

In [12]:
# working data question 2
mixtures = {
    "example": [
        {"component": "C\u2081", "name": "Methane", "z": 0.5301, "tc": -116.67, "pc": 666.4, "w": 0.0104},
        {"component": "n-C\u2084", "name": "n-Butane", "z": 0.1055, "tc": 305.63, "pc": 550.6, "w": 0.1995},
        {"component": "n-C\u2081\u2080", "name": "n-Decane", "z": 0.3644, "tc": 652.03, "pc": 305.2, "w": 0.4894},
    ],
    "assignment": [
        {"component": "C\u2081", "name": "Methane", "z": 0.5523, "tc": -116.67, "pc": 666.4, "w": 0.0104},
        {"component": "n-C\u2084", "name": "n-Butane", "z": 0.3630, "tc": 305.63, "pc": 550.6, "w": 0.1995},
        {"component": "n-C\u2081\u2080", "name": "n-Decane", "z": 0.0838, "tc": 652.03, "pc": 305.2, "w": 0.4894}
    ]
}

### Calculate the coefficients of the components in the mixture

In [23]:
# calculate_xj of each component
def calculate_xj(z, k, ng):
    return z / (1 + (ng * (k - 1)))

def caculate_yj(z, k, nl):
    return z / (1 + (nl * ((1/kj) - 1)))

def sum_of_a_key_in_list_of_dict(key, list):
    total_sum = 0
    for item in list:
        total_sum += item[key]
    return total_sum


In [ ]:
# The currently selected mixture and its default
current_mixture = []

title_2 = widgets.HTML(
    value="<h2>Select the mixture and input the temperature</h2>"
)
display(title_2)

temperature_2 = widgets.FloatText(
    value="160", 
    description="Farenheit", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)


mixture_dropdown_options = [(value, index) for index, value in enumerate(mixtures.keys())]
# mixture dropdown
mixture_dropdown = widgets.Dropdown(
    options=mixture_dropdown_options,
    value=0,
    description='Mixture', 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

ng_widget = widgets.FloatText(
    value="0.0001", 
    description="ng", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

# define the default for the current selected mixture from the dropdown
current_mixture = mixtures[list(mixtures.keys())[mixture_dropdown.value]]
    
# dynamic widgets
dyn_widget_dict = {f"{component['component']}": widgets.FloatText(value="0.0001", description=f"k ({component['component']})", layout={'width': '25%', 'margin': '10px 0px 10px 10px'}) for component in current_mixture}
dyn_widget_args = [dyn_widget_dict[key] for key in dyn_widget_dict]


def on_mixture_temp_change(mixture, sys_temp, ng, **args):
    current_mixture = mixtures[list(mixtures.keys())[mixture]]
    step_2_data = []
    
    for component in current_mixture:
        pc = component['pc']
        tc = component['tc']
        w = component['w']
        tr = round(calculate_tr(sys_temp, tc), 5)
        component['tr'] = tr
        b = round(calculate_b(tc, pc), 4)
        component['b'] = b
        ac = round(calculate_ac(tc, pc), 0)
        component['ac'] = ac
        alpha = round(calculate_alpha(w, sys_temp, tc), 4)
        component['alpha'] = alpha
        aT = round(calculate_at(ac, alpha), 0)
        component['aT'] = aT
        
        #  add the k values from the widgets to the data
        component['K'] = args[component['component']]
        short_dict = {
            "component": component['component'],
            "K": component['K'],
            "z": component['z']
        }
        step_2_data.append(short_dict)
    
#     render_table(current_mixture, "PR-EOS REQUIRED COEFFICIENTS", False, True, 'component')
    step_2(step_2_data)

def step_2(step_2_data):
    returned_df = render_table(
            step_2_data, 
            "Composition Table", 
            transpose=False, 
            change_indexing_id=True, 
            new_index='component', 
            should_display=False
    )
    
    
    # get the k values
    initial_ks = returned_df['K'].to_dict()
    
    print(returned_df.columns)
    # the iteration loops
    for kc1 in np.arange(initial_ks['C\u2081'], 500.0001, 0.0001):
        # update cell
        returned_df.loc['C\u2081', 'K'] = kc1
        for kc4 in np.arange(initial_ks['n-C\u2084'], 100.0001, 0.0001):
            returned_df.loc['n-C\u2084', 'K'] = kc4
            for kc10 in np.arange(initial_ks['n-C\u2081\u2080'], 10.0001, 0.0001):
                returned_df.loc['n-C\u2081\u2080', 'K'] = kc10
                
                for loop_ng in np.arange(ng_widget.value, 1.0001, 0.0001):
                    
                    returned_df['xj'] = returned_df.apply(lambda row: calculate_xj(int(row['z']), int(row['K']), loop_ng, axis=1))
                    returned_df.loc['Totals'] = ['', '', returned_df['xj'].sum()]
                    
                    if math.isclose(returned_df.loc['Totals', 'xj'], 1, rel_tol=0.0001):
                        # stop iterating once a specific outcome is reached
                        break
                else:
                    continue
                break
            else:
                continue
            break
        else:
            continue
        break
    
    
    display(returned_df)
    
interactive_widgets = widgets.interactive(
    on_mixture_temp_change, 
    mixture=mixture_dropdown, 
    sys_temp=temperature_2,
    ng=ng_widget,
    **dyn_widget_dict
)
for arg in dyn_widget_args:
    arg.observe(lambda change: interactive_widgets.update(), 'value')

display(interactive_widgets)


HTML(value='<h2>Select the mixture and input the temperature</h2>')

interactive(children=(Dropdown(description='Mixture', layout=Layout(margin='10px 0px 10px 10px', width='25%'),…